# Settings

In [1]:
options(repr.plot.width = 5, repr.plot.height = 3, scipen = 999, warn = -1)

suppressMessages(library(dplyr))
suppressMessages(library(lubridate))
library(xml2)
library(rvest)
library(curl)
library(stringr)
library(RCurl)
suppressMessages(library(httr))
library(rjson)

user_agents <- read.csv('user_agents.txt', stringsAsFactors = F, row.names = NULL, sep = '*') %>% pull(user_agents)

Loading required package: bitops


# Visa free countries (egov)

In [2]:
# countries w/o visas for KZ
visa_free <- 'https://egov.kz/cms/en/articles/move_abroad/visa_free_regime'

visa_free_df <- 
    html(visa_free) %>% 
    html_nodes('table') %>% 
    html_table(fill = T, header = T) %>% 
    as.data.frame

colnames(visa_free_df) <- visa_free_df[1, ]

visa_free_df <- 
    visa_free_df[-1, ] %>% 
    select(Country, N) %>% 
    mutate(N = str_trim(N),
          Country = str_trim(Country)) %>% 
    filter(N != 'visa') %>% 
    mutate(N = str_remove(N, 'during.*'),
           duration = str_extract(N, '(?<=until)(.*)(?=days)'),
           duration = as.numeric(duration)) %>% 
    filter(!is.na(duration)) %>% 
    select(-N) %>%
    select_all(tolower) %>% 
    mutate(country = recode(country, Korea = 'South Korea', Argentine = 'Argentina', UAE = 'United Arab Emirates')) %>% 
    arrange(-duration)

visa_free_df %>% head

country,duration
<chr>,<dbl>
Georgia,365
Armenia,180
Azerbaijan,90
Albania,90
Belarus,90
Colombia,90


# Capitals (wiki)
## Making major dataframe countries_df

In [3]:
# capitals
wiki_capitals <- 'https://en.wikipedia.org/wiki/List_of_national_capitals'

wiki_capitals_df <- 
    html(wiki_capitals) %>% 
    html_nodes('.wikitable') %>% 
    html_table() %>% 
    as.data.frame %>% 
    select_all(tolower) %>% 
    select(country, capital = city) %>% 
    mutate(capital = str_remove(capital, '\\(.*'),
           capital = str_trim(capital),
           country = str_trim(country))

kz_data <- data.frame(country = 'Kazakhstan', duration = 365, capital = 'Almaty')

countries_df <- 
visa_free_df %>%
left_join(wiki_capitals_df) %>% 
rbind(kz_data) %>% 
mutate(capital = case_when(country == 'Colombia' ~ 'Bogota',
                           country == 'Moldova' ~ 'Chisinau',
                           country == 'Hong Kong' ~ 'Hong Kong',
                           country == 'Costa Rica' ~ 'San Jose',
                           country == 'Brazil' ~ 'Brasilia',
                           TRUE ~ capital),
       country_html = str_replace_all(country, ' ', '%20'),
       capital_html = str_replace_all(capital, ' ', '%20'))

countries_df %>% tail

Joining, by = "country"


,country,duration,capital,country_html,capital_html
,<chr>,<dbl>,<chr>,<chr>,<chr>
24,Philippines,30,Manila,Philippines,Manila
25,Montenegro,30,Podgorica,Montenegro,Podgorica
26,Ecuador,30,Quito,Ecuador,Quito
27,Barbados,28,Bridgetown,Barbados,Bridgetown
28,Hong Kong,14,Hong Kong,Hong%20Kong,Hong%20Kong
29,Kazakhstan,365,Almaty,Kazakhstan,Almaty


# Prices comparison (Numbeo)

In [4]:
# prices
Amount = '250000'

numbeoUrl <- function(country, city, amount = Amount){
    paste0('https://www.numbeo.com/cost-of-living/compare_cities.jsp?country1=Kazakhstan&city1=Almaty&country2=',
           country,
           '&city2=',
           city,
           '&amount=',
           amount,
           '&displayCurrency=KZT')
}

getPrices <- function(country, city, amount) {
    
    #Sys.sleep(15)
    
    tryCatch(
        numbeoUrl(country, city, amount = Amount) %>% 
        curl(., handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
        html %>% 
        html_nodes('.light_box') %>% 
        html_text %>% 
        str_extract('You would need around [0-9]+,[0-9]+') %>% 
        str_remove_all('[^[:digit:]]') %>% 
        as.numeric(),
        
        error = function(err){NA}
    )
}

#numbeoUrl('Georgia', 'Tbilisi')
#getPrices('Georgia', 'Tbilisi')

In [5]:
prices_df <-
    countries_df %>% 
    arrange(-duration) %>% 
    mutate(prices = Vectorize(getPrices)(country_html, capital_html),
          prices = as.numeric(prices),
          prices_difference = prices / as.numeric(Amount) - 1)

prices_df %>% head

country,duration,capital,country_html,capital_html,prices,prices_difference
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Georgia,365,Tbilisi,Georgia,Tbilisi,235296,-0.058816
Kazakhstan,365,Almaty,Kazakhstan,Almaty,250000,0.000000
Armenia,180,Yerevan,Armenia,Yerevan,271971,0.087884
Azerbaijan,90,Baku,Azerbaijan,Baku,253643,0.014572
Albania,90,Tirana,Albania,Tirana,301698,0.206792
Belarus,90,Minsk,Belarus,Minsk,297496,0.189984


# Internet Speed (speedtest.net)

In [6]:
# Internet

speedtest <- 'https://www.speedtest.net/global-index'

getSpeed <- function(divType = '#column-mobile', NAME = 'mobileSpeed') {
    html(speedtest) %>% 
    html_nodes(divType) %>% 
    html_nodes('table') %>% 
    html_table(fill = T, header = T) %>% 
    as.data.frame %>% 
    filter(!is.na(Mbps)) %>% 
    select(3, 4) %>% 
    select_all(tolower) %>% 
    mutate(country = str_trim(str_remove(country, '\\(.*'))) %>% 
    setNames(c('country', NAME))
}

speed_df <- 
    countries_df %>% 
    left_join(getSpeed()) %>% 
    left_join(getSpeed('#column-fixed', 'fixedSpeed'))

speed_df %>% head

Joining, by = "country"
Joining, by = "country"


country,duration,capital,country_html,capital_html,mobileSpeed,fixedSpeed
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
Georgia,365,Tbilisi,Georgia,Tbilisi,27.14,24.87
Armenia,180,Yerevan,Armenia,Yerevan,31.01,25.61
Azerbaijan,90,Baku,Azerbaijan,Baku,31.84,18.54
Albania,90,Tirana,Albania,Tirana,49.21,30.83
Belarus,90,Minsk,Belarus,Minsk,12.59,50.14
Colombia,90,Bogota,Colombia,Bogota,18.12,23.88


# Time difference with Kiyv and Almaty (worldtimeapi)

In [7]:
# timezone
timezones <- 
    read.csv('http://worldtimeapi.org/api/timezone.txt', header = F, stringsAsFactors = F) %>%
    setNames('tzName') %>% 
    mutate(city = str_extract(tzName, '\\/.*'),
           city = str_remove(city, '\\/'),
           city = ifelse(str_detect(city, '\\/'), str_extract(city, '\\/.*'), city),
           city = str_remove(city, '\\/'),
           city = str_trim(city))

worldtime <- 'http://worldtimeapi.org/api/timezone/'

timezones_df <- 
    countries_df %>% 
    select(capital) %>% 
    mutate(capital_corr = str_replace_all(capital, ' ', '_')) %>% 
    left_join(timezones, by = c('capital_corr' = 'city')) %>% 
    select(-capital_corr) %>% 
    mutate(tzName = case_when(capital %in% c('Tirana', 'Podgorica') ~ 'Europe/Belgrade',
                              capital == 'San Jose' ~ 'America/Mexico_City',
                              capital == 'Brasilia' ~ 'America/Argentina/Buenos_Aires',
                              capital == 'Ankara' ~ 'Europe/Moscow',
                              capital == 'Abu Dhabi' ~ 'Asia/Dubai',
                              capital == 'Quito' ~ 'America/Bogota',
                              capital == 'Bridgetown' ~ 'America/Puerto_Rico',
                              TRUE ~ tzName),
           tzUrl = paste0(worldtime, tzName, '.json')) 

tzUnique <- timezones_df$tzUrl %>% unique

In [8]:
# # Save WorldTime data bc it takes time to import

# tzAPI <- read.csv(text = 'tzUrl, utc_datetime, utc_offset', stringsAsFactors = F)

# for (i in 1:length(tzUnique)) {
    
#     Sys.sleep(3)
    
#     url <- tzUnique[i]
    
#     tzInfo <- url %>% fromJSON(file = .) 
    
#     tzAPI <- rbind(tzAPI, data.frame(tzUrl = url, utc_datetime = tzInfo$datetime, utc_offset = tzInfo$utc_offset))    
    
# }

# write.csv(tzAPI, 'tzAPI.csv', row.names = F)

In [9]:
tzImport <- read.csv('tzAPI.csv')

almaty_time <- tzImport %>% filter(str_detect(tzUrl, 'Almaty')) %>% pull(utc_datetime) %>% str_sub(1, 19) %>% ymd_hms(.)
kiyv_time <- tzImport %>% filter(str_detect(tzUrl, 'Kiev')) %>% pull(utc_datetime) %>% str_sub(1, 19) %>% ymd_hms(.)

tz_df <- 
    timezones_df %>% 
    left_join(tzImport) %>% 
    mutate(dt = ymd_hms(str_sub(utc_datetime, 1, 19)),
          almaty_difference = round(interval(almaty_time, dt) / hours(1)),
          kiyv_difference = round(interval(kiyv_time, dt) / hours(1))) %>% 
    select(capital, contains('diff'))

tz_df %>% head

Joining, by = "tzUrl"


capital,almaty_difference,kiyv_difference
<chr>,<dbl>,<dbl>
Tbilisi,-2,2
Yerevan,-2,2
Baku,-2,2
Tirana,-5,-1
Minsk,-3,1
Bogota,-11,-7


# Air Pollution
# Two sources: aqicn.org and air-quality.com

In [10]:
# trying one source, cancelled because of restrictions and small cities db

# airVisCountries <- 
#     fromJSON(file = 'http://api.airvisual.com/v2/countries?key=c19c5115-563a-42e3-b0e0-a6ba154e52fa')$data %>% 
#     unlist(., use.names = F) %>% 
#     data.frame(country = ., stringsAsFactors = F)

# airVisCountries %>% head

In [11]:
getAQI <- function(city) {
    
    tryCatch(
        
        fromJSON(file = paste0('https://api.waqi.info/feed/', city, '/?token=4989c7b86b7d3dbd7a5966ea20610cc6e5e36cd1'))$data$aqi,
        
        error = function(err){NA}
    )    
}

getAQI('odessa')

AQI_df <- 
    countries_df %>% 
    mutate(AQI = Vectorize(getAQI)(capital))

[1] 87

In [12]:
AQcom <- 'https://air-quality.com/places'

# get country links
AQlinks <- 
    curl(AQcom, handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
    html %>% 
    html_nodes('.country-item') %>% 
    html_attr('href') %>% 
    str_trim

# make countries df
AQcom_df <- 
    data.frame(url = AQlinks, stringsAsFactors = F) %>% 
    mutate(country = str_extract(url, '(?<=country/)(.*)(?=/)'),
           country = str_to_title(country),
           country = str_trim(country)) %>% 
    inner_join(filter(AQI_df, is.na(AQI) | AQI == '-'))

# add city info 
AQ_city_df <- read.csv(text = 'city, city_url', stringsAsFactors = F)

for (i in 1:nrow(AQcom_df)) {
    
    AQ_city <- 
        AQcom_df$url[i] %>% 
        curl(., handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
        html %>% 
        html_nodes('.main-block-content') %>% 
        html_nodes('.site-item')

    AQ_city_df <- 
        data.frame(
            city = str_trim(AQ_city %>% html_nodes('.title') %>% html_text),
            city_url = str_trim(AQ_city %>% html_attr('href')),
        stringsAsFactors = F) %>% 
        rbind(AQ_city_df, .)
    
}

# get AQI from city link
getAQIv2 <- function(city_url) {
    
    tryCatch(
        
         curl(city_url, handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
        html %>% 
        html_nodes('.indexValue') %>% 
        html_text %>% 
        as.numeric(),
        
        error = function(err){NA}
    )   
}

# create df for empty cities
AQI_df_v2 <- 
    AQ_city_df  %>% 
    mutate(city = str_replace(city, 'North Governorate', 'Beirut'),
           city = str_remove_all(tolower(city), 'province|region|voblasc|county|metro'),
           city = str_trim(city),
           city = str_to_title(city)) %>% 
    right_join(AQcom_df, by = c('city' = 'capital')) %>% 
    mutate(AQI = Vectorize(getAQIv2)(city_url)) %>% 
    select(capital = city, AQI2 = AQI)

head(AQI_df_v2)

Joining, by = "country"


capital,AQI2
<chr>,<dbl>
Baku,53
Tbilisi,NA
Almaty,108
Almaty,74
Beirut,77
Manila,47


In [13]:
AQI_both_df <- 
    AQI_df %>% 
    left_join(AQI_df_v2) %>% 
    mutate(AQI = ifelse(is.na(AQI) | AQI == '-', AQI2, AQI),
           AQI = as.numeric(AQI),
           AQI = case_when(is.na(AQI) & capital == 'Yerevan' ~ 74,
                         is.na(AQI) & capital == 'Chisinau' ~ 26,
                         is.na(AQI) & capital == 'Podgorica' ~ 97,
                         TRUE ~ AQI))  %>% 
    select(-AQI2) %>% 
    select(country, AQI)

head(AQI_both_df)

Joining, by = "capital"


country,AQI
<chr>,<dbl>
Georgia,NA
Armenia,74
Azerbaijan,53
Albania,NA
Belarus,22
Colombia,9


# Climate (weather-atlas.com)

In [14]:
# temperature

# getAtlas <- function(country, city) {
    
#     paste0('https://www.weather-atlas.com/en/', country, '/', city, '-climate')
# }

## CHECK if all cities are there

# for (i in 1:nrow(countries_df)) {
    
#     atlasUrl <- 
#         getAtlas(str_replace_all(countries_df$country[i], ' ', '-'), str_replace_all(countries_df$capital[i], ' ', '-')) %>% 
#         curl(., handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
#         html

#     atlasName <- 
#         atlasUrl %>% 
#         html_nodes('[itemprop = addressLocality]') %>% 
#         html_text

#     if (countries_df$capital[i] == atlasName) {

#         print('OK')
        
#     } else {

#         print('Not Found')
#         print(countries_df$capital[i])
#     }
        
# }

## NAMES ARE OK

In [15]:
# temperature

getAtlas <- function(country, city) {
    
    paste0('https://www.weather-atlas.com/en/', country, '/', city, '-climate')
}

climate_df <- read.csv(text = 'month, metric, value, capital', stringsAsFactors = F)


for (i in 1:nrow(countries_df)) {
    
    country <- str_replace_all(countries_df$country[i], ' ', '-')
    capital <- str_replace_all(countries_df$capital[i], ' ', '-') 
    
    #print(c(country, capital))
    
    plotScript <- 
        getAtlas(country, capital) %>% 
        curl(., handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
        html %>% 
        html_nodes('script') %>% 
        html_text

    plotTitles <- 
        plotScript %>% 
        str_extract('(?<=title:)(.*)(?=\\,)') %>% 
        str_remove_all("\\'|\\(\\[") %>% 
        .[!is.na(.)]

    plotData <- 
        plotScript %>% 
        str_extract('(?<=data.addRows)(.*)(?=\\]\\)\\;)') %>% 
        str_remove_all("\\'|\\(\\[") %>% 
        .[!is.na(.)]
    
    for (j in 1:length(plotTitles)) {
    
        climate_df <- 
            plotData[j] %>% 
            strsplit(split='],', fixed=TRUE) %>% 
            unlist %>% 
            str_remove_all('\\[') %>% 
            data.frame(x = ., stringsAsFactors = F)  %>% 
            tidyr::separate(x, c('month', plotTitles[j]), sep = ',', remove = T) %>% 
            mutate_at(vars(contains(plotTitles[j])), ~as.numeric(.)) %>% 
            tidyr::gather(metric, value, -month) %>% 
            mutate(capital = countries_df$capital[i]) %>% 
            rbind(., climate_df)
    }
    
}

climate_df %>% pull(metric) %>% unique %>% sort

[1] "Daylight hours / Sunshine hours" "Humidity (%)"                   
[3] "Rainfall (mm)"                   "Rainfall days"                  
[5] "Sea temperature (°C)"            "Snowfall days"                  
[7] "Temperature (°C)"                "UV index"

In [16]:
climate_df %>% head

month,metric,value,capital
<chr>,<chr>,<dbl>,<chr>
Jan,UV index,1,Almaty
Feb,UV index,2,Almaty
Mar,UV index,4,Almaty
Apr,UV index,6,Almaty
May,UV index,8,Almaty
Jun,UV index,9,Almaty


In [17]:
climate_summary <- 
    climate_df %>% 
    group_by(capital, metric) %>% 
    summarise(value = round(mean(value), 2)) %>% 
    ungroup %>% 
    tidyr::spread(metric, value)

# Safety
## Global Peace Index 2019

In [18]:
# safety

safety <- 'http://worldpopulationreview.com/countries/safest-countries-in-the-world/'

safety_df <- 
    html(safety) %>% 
    html_nodes('table') %>% 
    html_table(header = T) %>% 
    as.data.frame %>% 
    select_all(tolower) %>% 
    select(country, peaceIndex = `peace.index`)

countries_df %>% 
    left_join(safety_df) %>% 
    filter(is.na(peaceIndex))

Joining, by = "country"


country,duration,capital,country_html,capital_html,peaceIndex
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Kyrgyzstan,90,Bishkek,Kyrgyzstan,Bishkek,NA
Barbados,28,Bridgetown,Barbados,Bridgetown,NA
Hong Kong,14,Hong Kong,Hong%20Kong,Hong%20Kong,NA


# Cigarettes consumption

In [19]:
# сижки
cigarettes <- 'https://en.wikipedia.org/wiki/List_of_countries_by_cigarette_consumption_per_capita'

cigarettes_df <- 
    html(cigarettes) %>% 
    html_nodes('.wikitable') %>% 
    html_table(fill = T, header = T) %>% 
    as.data.frame() %>% 
    select_all(tolower) %>% 
    select(country, cigConsumption = `var.3`) %>% 
    mutate(country = recode(country, 'UAE' = 'United Arab Emirates'))

countries_df %>% 
left_join(cigarettes_df) %>% 
filter(is.na(cigConsumption))

Joining, by = "country"


country,duration,capital,country_html,capital_html,cigConsumption
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Hong Kong,14,Hong Kong,Hong%20Kong,Hong%20Kong,NA


# Pollution Index from Numbeo

In [20]:
pollution_index <- 'https://www.numbeo.com/pollution/rankings_by_country.jsp?title=2019-mid'

pollution <- pollution_index %>% 
    curl(., handle = curl::new_handle('useragent' = sample(user_agents, 1))) %>% 
    html %>% 
    html_nodes('#t2') %>% 
    html_table(fill = T, header = T) %>% 
    as.data.frame %>% 
    select(country = Country, Pollution.Index)

countries_df %>% 
left_join(pollution) %>% 
filter(is.na(Pollution.Index))

Joining, by = "country"


country,duration,capital,country_html,capital_html,Pollution.Index
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Kyrgyzstan,90,Bishkek,Kyrgyzstan,Bishkek,NA
Moldova,90,Chisinau,Moldova,Chisinau,NA
Tajikistan,30,Dushanbe,Tajikistan,Dushanbe,NA
Uzbekistan,30,Tashkent,Uzbekistan,Tashkent,NA
Barbados,28,Bridgetown,Barbados,Bridgetown,NA


# Freedom on the Net

In [21]:
library(openxlsx)

freedom <- read.xlsx('FOTN2019_data.xlsx', sheet = 1, startRow = 2) %>% select(country = Country, freedom = Total)

countries_df %>% 
left_join(freedom) %>% 
filter(is.na(freedom))

Joining, by = "country"


country,duration,capital,country_html,capital_html,freedom
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Albania,90,Tirana,Albania,Tirana,NA
Costa Rica,90,San Jose,Costa%20Rica,San%20Jose,NA
Moldova,90,Chisinau,Moldova,Chisinau,NA
Mongolia,90,Ulaanbaatar,Mongolia,Ulaanbaatar,NA
Serbia,30,Belgrade,Serbia,Belgrade,NA
Tajikistan,30,Dushanbe,Tajikistan,Dushanbe,NA
Montenegro,30,Podgorica,Montenegro,Podgorica,NA
Barbados,28,Bridgetown,Barbados,Bridgetown,NA
Hong Kong,14,Hong Kong,Hong%20Kong,Hong%20Kong,NA


# All in One

In [22]:
# all in one

all_in_one <- 
    countries_df %>% 
    left_join(prices_df) %>% 
    left_join(speed_df) %>% 
    left_join(tz_df) %>% 
    left_join(AQI_both_df) %>% 
    left_join(climate_summary) %>% 
    left_join(safety_df) %>% 
    left_join(cigarettes_df) %>% 
    left_join(pollution) %>% 
    left_join(freedom) %>% 
    select(-contains('html')) %>% 
    select(country, capital, everything()) %>% 
    distinct(country, .keep_all = T)

write.csv(all_in_one, 'all_data.csv', sep = ',', row.names = F)

Joining, by = c("country", "duration", "capital", "country_html", "capital_html")
Joining, by = c("country", "duration", "capital", "country_html", "capital_html")
Joining, by = "capital"
Joining, by = "country"
Joining, by = "capital"
Joining, by = "country"
Joining, by = "country"
Joining, by = "country"
Joining, by = "country"


In [23]:
getTop <- function(colName, asc = TRUE, n = 5) {
    
    if (asc) {
        
        all_in_one %>% 
        arrange(!!sym(colName)) %>% 
        head(n)
        
    } else {
        
        all_in_one %>% 
        arrange(desc(!!sym(colName))) %>% 
        head(n)
        
    }   
    
}

all_in_one %>% head

country,capital,duration,prices,prices_difference,mobileSpeed,fixedSpeed,almaty_difference,kiyv_difference,AQI,...,Rainfall (mm),Rainfall days,Sea temperature (°C),Snowfall days,Temperature (°C),UV index,peaceIndex,cigConsumption,Pollution.Index,freedom
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Georgia,Tbilisi,365,235296,-0.058816,27.14,24.87,-2,2,NA,...,43.08,5.91,NA,NA,18.52,5.33,2.122,1917.7,71.55,75
Armenia,Yerevan,180,271971,0.087884,31.01,25.61,-2,2,74,...,24.67,6.50,NA,1.86,18.88,6.17,2.294,1985.7,60.94,76
Azerbaijan,Baku,90,253643,0.014572,31.84,18.54,-2,2,53,...,17.50,4.08,15.92,0.83,18.10,5.50,2.425,1525.6,79.36,39
Albania,Tirana,90,301698,0.206792,49.21,30.83,-5,-1,NA,...,105.42,10.67,NA,NA,20.93,5.08,1.821,2491.6,75.75,NA
Belarus,Minsk,90,297496,0.189984,12.59,50.14,-3,1,22,...,57.50,15.00,NA,8.53,10.64,2.92,2.115,2911.3,43.75,35
Colombia,Bogota,90,265948,0.063792,18.12,23.88,-11,-7,9,...,84.42,15.17,NA,NA,19.48,12.00,2.661,351.4,62.57,67


In [24]:
getTop('duration', F)

country,capital,duration,prices,prices_difference,mobileSpeed,fixedSpeed,almaty_difference,kiyv_difference,AQI,...,Rainfall (mm),Rainfall days,Sea temperature (°C),Snowfall days,Temperature (°C),UV index,peaceIndex,cigConsumption,Pollution.Index,freedom
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Georgia,Tbilisi,365,235296,-0.058816,27.14,24.87,-2,2,NA,...,43.08,5.91,NA,NA,18.52,5.33,2.122,1917.7,71.55,75
Kazakhstan,Almaty,365,250000,0.000000,19.03,38.09,0,4,108,...,57.00,10.17,NA,4.45,15.77,5.08,1.932,1800.9,75.42,32
Armenia,Yerevan,180,271971,0.087884,31.01,25.61,-2,2,74,...,24.67,6.50,NA,1.86,18.88,6.17,2.294,1985.7,60.94,76
Azerbaijan,Baku,90,253643,0.014572,31.84,18.54,-2,2,53,...,17.50,4.08,15.92,0.83,18.10,5.50,2.425,1525.6,79.36,39
Albania,Tirana,90,301698,0.206792,49.21,30.83,-5,-1,NA,...,105.42,10.67,NA,NA,20.93,5.08,1.821,2491.6,75.75,NA


In [25]:
write.table(all_in_one, 'all_data.txt', sep = '\t', row.names = F)
write.table(inner_join(climate_df, select(countries_df, country, capital)), 'climate.txt', sep = '\t', row.names = F)

Joining, by = "capital"
